In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt
from sklearn.model_selection import train_test_split


In [20]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

def visualize_distribution(df, columns=None):

    # Default columns if none are provided
    if columns is None:
        columns = list(df.columns)[:-1]
    
    # Check if all columns exist in the DataFrame
    missing_cols = [col for col in columns if col not in df.columns]
    if missing_cols:
        raise ValueError(f"Missing columns in DataFrame: {missing_cols}")
    
    # Set up the matplotlib figure - adjust the figure size based on number of columns
    plt.figure(figsize=(len(columns) * 5, 5))
    
    # Create a count plot for each column
    for i, column in enumerate(columns):
        plt.subplot(1, len(columns), i + 1)
        # Plotting with seaborn countplot
        ax = sns.countplot(y=column, data=df, order=df[column].value_counts().index)
        plt.title(f'Distribution of {column}')
        plt.xlabel('Percentage')
        plt.ylabel(column)
        
        # Calculate the total for percentages
        total = df[column].notnull().sum()
        # Add annotations
        for p in ax.patches:
            percentage = f'{100 * p.get_width() / total:.1f}%'
            x = p.get_x() + p.get_width() + 0.02  # slight offset to the right of the bar for readability
            y = p.get_y() + p.get_height() / 2
            ax.annotate(percentage, (x, y))
        
        # Remove the frame
        sns.despine(ax=ax)
    
    plt.tight_layout()
    plt.show()

### df is the time series dataframe, whole_df is the entire background dataset

In [3]:
df = pd.read_csv('time_series_thesis.csv')
whole_df = pd.read_csv('whole_dataset.csv')

In [4]:
df = df.drop(columns=df.columns[2:6])


In [5]:
# create a identifying firm-corridor combination
df['firm-corridor'] = df['firm'] + ' ' + df['corridor']

In [6]:
# to save which corridors are present
corridors_present = list(df['corridor'].unique())

In [7]:
len(corridors_present) # There are 199 corridors

199

In [8]:
df

,corridor,firm,2014-01-01 00:00:00,2014-04-01 00:00:00,2014-07-01 00:00:00,2014-10-01 00:00:00,2015-01-01 00:00:00,2015-04-01 00:00:00,2015-07-01 00:00:00,2015-10-01 00:00:00,...,2021-07-01 00:00:00,2021-10-01 00:00:00,2022-01-01 00:00:00,2022-04-01 00:00:00,2022-07-01 00:00:00,2022-10-01 00:00:00,2023-01-01 00:00:00,2023-04-01 00:00:00,2023-07-01 00:00:00,firm-corridor
0,AREEGY,Al Ansari,3.940,3.575,3.91,3.920,3.82,3.770,4.010,3.240,...,2.973333,3.600,3.480,3.150000,3.090000,2.470000,2.970000,2.390000,2.180000,Al Ansari AREEGY
1,AREEGY,Al Fardan Exchange,4.055,4.625,4.34,3.885,4.92,4.775,5.950,5.560,...,11.636667,12.130,16.445,3.060000,2.903333,3.320000,2.730000,2.336667,2.530000,Al Fardan Exchange AREEGY
2,AREEGY,MoneyGram,4.470,2.780,2.78,2.140,3.51,3.510,3.620,3.600,...,3.580000,3.560,3.550,5.840000,4.860000,5.690000,5.160000,3.690000,3.690000,MoneyGram AREEGY
3,AREEGY,Wall St Exchange,3.410,4.440,3.58,2.900,3.34,3.290,4.010,3.240,...,5.320000,3.960,4.010,3.450000,3.080000,3.050000,2.780000,2.680000,2.650000,Wall St Exchange AREEGY
4,AREEGY,Western Union,5.860,5.770,5.77,5.130,6.50,7.790,7.890,7.880,...,3.400000,4.030,3.980,3.700000,3.730000,3.655000,3.720000,3.765000,3.665000,Western Union AREEGY
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
769,ZAFZWE,First National Bank of South Africa,19.210,18.530,18.97,13.060,12.75,13.085,13.245,18.105,...,26.725000,26.855,27.220,29.083333,32.695000,35.586667,35.803333,44.953333,15.773333,First National Bank of South Africa ZAFZWE
770,ZAFZWE,Mukuru,15.495,13.565,10.95,18.320,19.24,10.770,11.810,10.000,...,10.475000,9.065,8.905,11.050000,9.320000,10.423333,10.141111,8.616429,11.991429,Mukuru ZAFZWE
771,ZAFZWE,Nedbank,18.770,18.790,18.78,18.250,18.34,19.880,17.530,18.280,...,15.400000,15.180,25.460,25.370000,25.520000,25.450000,30.680000,22.840000,30.790000,Nedbank ZAFZWE
772,ZAFZWE,Standard Bank,18.460,18.200,18.46,18.570,18.32,19.350,18.430,17.420,...,13.910000,13.660,25.230,25.260000,24.520000,25.070000,27.370000,27.520000,26.070000,Standard Bank ZAFZWE


In [9]:
# a dataframe that gets the number of banks changing with time
unique_bank_df = whole_df.groupby(by=['corridor', 'period'])['firm'].agg('nunique')
unique_bank_df = unique_bank_df.reset_index()
unique_bank_df = unique_bank_df[unique_bank_df['corridor'].isin(corridors_present)]

In [10]:
whole_df['firm-corridor'] = whole_df['firm'] + ' ' + whole_df['corridor']

In [ ]:
# region, G8/20 membership
country_characters = whole_df[['source_region', 'source_G8G20', 'destination_region',
       'destination_G8G20', 'corridor']]
country_characters.drop_duplicates(inplace=True)
country_characters = country_characters[country_characters['corridor'].isin(corridors_present)]

In [12]:


country_characters['source_income'] = country_characters['source_income'].replace({
    'High income: OECD': 'High income',
    'High income: nonOECD': 'High income',
    'High income': 'High income'
})

print(country_characters['source_income'].unique())


['High income' 'Upper middle income' 'Lower middle income' 'Low income']


/var/folders/k8/ltw6mcfj23ndd4m6h12lm7fh0000gn/T/ipykernel_76618/972671726.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  country_characters['source_income'] = country_characters['source_income'].replace({


In [13]:
country_characters['destination_income'] = country_characters['destination_income'].replace({
    'High income: OECD': 'High income',
    'High income: nonOECD': 'High income',
    'High income': 'High income',
    '..': 'High income' # Netherlands Antilles
})

/var/folders/k8/ltw6mcfj23ndd4m6h12lm7fh0000gn/T/ipykernel_76618/1662405052.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  country_characters['destination_income'] = country_characters['destination_income'].replace({


In [14]:


country_characters['source_region'] = country_characters['source_region'].replace('..', 'the West')
country_characters['source_G8G20'] = country_characters['source_G8G20'].replace('..', 'Not G20')
country_characters['destination_region'] = country_characters['destination_region'].replace('..', 'North America')
country_characters['destination_G8G20'] = country_characters['destination_G8G20'].replace('..', 'Non G20')

/var/folders/k8/ltw6mcfj23ndd4m6h12lm7fh0000gn/T/ipykernel_76618/2227919566.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  country_characters['source_region'] = country_characters['source_region'].replace('..', 'the West')
/var/folders/k8/ltw6mcfj23ndd4m6h12lm7fh0000gn/T/ipykernel_76618/2227919566.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  country_characters['source_G8G20'] = country_characters['source_G8G20'].replace('..', 'Not G20')
/var/folders/k8/ltw6mcfj23ndd4m6h12lm7fh0000gn/T/ipykernel_

In [15]:
# Remove duplicate rows
country_characters = country_characters.drop_duplicates()


In [16]:
# Assuming 'corridor' is the name of the column you're interested in
country_characters = country_characters.drop_duplicates(subset='corridor', keep='last')


# Print the DataFrame to verify the duplicates have been removed
print(country_characters)

              source_income   destination_income               source_region  \
0               High income  Upper middle income                    the West   
20              High income  Lower middle income                    the West   
37              High income  Lower middle income                    the West   
64              High income  Lower middle income                    the West   
103             High income  Lower middle income                    the West   
...                     ...                  ...                         ...   
197667          High income  Upper middle income                    the West   
197864  Lower middle income  Upper middle income  Middle East & North Africa   
197891  Lower middle income           Low income  Middle East & North Africa   
198459          High income          High income                    the West   
199408          High income          High income                    the West   

       source_G8G20          destinatio

In [ ]:
country_characters.to_csv("country_characters.csv")

### showcasing the dataframes I have now

In [159]:
df

,corridor,firm,2014-01-01 00:00:00,2014-04-01 00:00:00,2014-07-01 00:00:00,2014-10-01 00:00:00,2015-01-01 00:00:00,2015-04-01 00:00:00,2015-07-01 00:00:00,2015-10-01 00:00:00,...,2021-07-01 00:00:00,2021-10-01 00:00:00,2022-01-01 00:00:00,2022-04-01 00:00:00,2022-07-01 00:00:00,2022-10-01 00:00:00,2023-01-01 00:00:00,2023-04-01 00:00:00,2023-07-01 00:00:00,firm-corridor
0,AREEGY,Al Ansari,3.940,3.575,3.91,3.920,3.82,3.770,4.010,3.240,...,2.973333,3.600,3.480,3.150000,3.090000,2.470000,2.970000,2.390000,2.180000,Al Ansari AREEGY
1,AREEGY,Al Fardan Exchange,4.055,4.625,4.34,3.885,4.92,4.775,5.950,5.560,...,11.636667,12.130,16.445,3.060000,2.903333,3.320000,2.730000,2.336667,2.530000,Al Fardan Exchange AREEGY
2,AREEGY,MoneyGram,4.470,2.780,2.78,2.140,3.51,3.510,3.620,3.600,...,3.580000,3.560,3.550,5.840000,4.860000,5.690000,5.160000,3.690000,3.690000,MoneyGram AREEGY
3,AREEGY,Wall St Exchange,3.410,4.440,3.58,2.900,3.34,3.290,4.010,3.240,...,5.320000,3.960,4.010,3.450000,3.080000,3.050000,2.780000,2.680000,2.650000,Wall St Exchange AREEGY
4,AREEGY,Western Union,5.860,5.770,5.77,5.130,6.50,7.790,7.890,7.880,...,3.400000,4.030,3.980,3.700000,3.730000,3.655000,3.720000,3.765000,3.665000,Western Union AREEGY
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
769,ZAFZWE,First National Bank of South Africa,19.210,18.530,18.97,13.060,12.75,13.085,13.245,18.105,...,26.725000,26.855,27.220,29.083333,32.695000,35.586667,35.803333,44.953333,15.773333,First National Bank of South Africa ZAFZWE
770,ZAFZWE,Mukuru,15.495,13.565,10.95,18.320,19.24,10.770,11.810,10.000,...,10.475000,9.065,8.905,11.050000,9.320000,10.423333,10.141111,8.616429,11.991429,Mukuru ZAFZWE
771,ZAFZWE,Nedbank,18.770,18.790,18.78,18.250,18.34,19.880,17.530,18.280,...,15.400000,15.180,25.460,25.370000,25.520000,25.450000,30.680000,22.840000,30.790000,Nedbank ZAFZWE
772,ZAFZWE,Standard Bank,18.460,18.200,18.46,18.570,18.32,19.350,18.430,17.420,...,13.910000,13.660,25.230,25.260000,24.520000,25.070000,27.370000,27.520000,26.070000,Standard Bank ZAFZWE


In [18]:
# Assuming `df` is your DataFrame

# Drop the first two columns and then make the last column the first one
columns = df.columns.tolist()  # Get the list of all column names
df_transposed = df[[columns[-1]] + columns[2:-1] ] .transpose() # Create a new order list & Reindex the DataFrame with the new order



In [21]:
df_transposed.head()

firm-corridor,Al Ansari AREEGY,Al Fardan Exchange AREEGY,MoneyGram AREEGY,Wall St Exchange AREEGY,Western Union AREEGY,Al Ansari AREIND,Al Fardan Exchange AREIND,Lari AREIND,MoneyGram AREIND,Wall St Exchange AREIND,...,MoneyGram ZAFZMB,Nedbank ZAFZMB,Standard Bank ZAFZMB,Western Union ZAFZMB,ABSA ZAFZWE,First National Bank of South Africa ZAFZWE,Mukuru ZAFZWE,Nedbank ZAFZWE,Standard Bank ZAFZWE,Western Union ZAFZWE
2014-01-01,3.94,4.055,4.47,3.41,5.86,2.545,2.516667,2.63,3.04,2.605,...,13.23,21.94,21.97,16.41,15.32,19.21,15.495,18.77,18.46,16.41
2014-04-01,3.575,4.625,2.78,4.44,5.77,2.54,2.576667,2.743333,2.91,2.725,...,12.27,22.19,20.69,15.12,15.39,18.53,13.565,18.79,18.2,15.12
2014-07-01,3.91,4.34,2.78,3.58,5.77,2.875,2.823333,2.863333,2.78,2.875,...,15.97,21.62,21.32,11.09,15.51,18.97,10.95,18.78,18.46,11.09
2014-10-01,3.92,3.885,2.14,2.9,5.13,2.615,3.093333,3.526667,2.91,3.925,...,13.02,23.26,23.52,15.92,14.79,13.06,18.32,18.25,18.57,15.92
2015-01-01,3.82,4.92,3.51,3.34,6.5,2.67,2.683333,3.23,3.06,2.905,...,11.34,20.48,21.28,15.98,15.97,12.75,19.24,18.34,18.32,15.98


In [20]:
# Set the column names to the values of the first row
df_transposed.columns = df_transposed.iloc[0]

# Then drop the first row as it's now redundant
df_transposed = df_transposed[1:]
df_transposed.index = pd.to_datetime(df_transposed.index)




In [22]:
# Identify columns where all rows but the last 6 are constant
constant_columns = df_transposed.iloc[:-6].nunique() == 1

# Filter out these columns
df_filtered = df_transposed.loc[:, ~constant_columns]

In [23]:
#df_transposed.to_csv('time_series_thesis_question_1.csv', index=False)
df_filtered.to_csv('time_series_thesis_question_1.csv', index=True, date_format='%Y-%m-%d')


In [164]:
whole_df

,id,period,source_name,source_region,source_income,source_lending,source_G8G20,destination_name,destination_region,destination_income,...,speed actual,cc1 lcu amount,cc1 denomination amount,cc1 lcu code,cc1 total cost %,inter lcu bank fx,transparent,date,corridor,firm-corridor
0,13123,2011_1Q,Australia,..,High income: OECD,..,G20,Fiji,East Asia & Pacific,Upper middle income,...,Less than one hour,200.0,200.0,AUD,8.79,1.83024,yes,24/Jan/2011,AUSFJI,Coinstar AUSFJI
1,13129,2011_1Q,Australia,..,High income: OECD,..,G20,Fiji,East Asia & Pacific,Upper middle income,...,Less than one hour,200.0,200.0,AUD,9.42,1.83024,yes,24/Jan/2011,AUSFJI,MoneyGram AUSFJI
2,13131,2011_1Q,Australia,..,High income: OECD,..,G20,Fiji,East Asia & Pacific,Upper middle income,...,3-5 days,200.0,200.0,AUD,11.00,1.00000,no,24/Jan/2011,AUSFJI,National Australia Bank (NAB) AUSFJI
3,13128,2011_1Q,Australia,..,High income: OECD,..,G20,Fiji,East Asia & Pacific,Upper middle income,...,Same day,200.0,200.0,AUD,11.88,1.83024,yes,24/Jan/2011,AUSFJI,IMEX Money Transfer AUSFJI
4,13127,2011_1Q,Australia,..,High income: OECD,..,G20,Fiji,East Asia & Pacific,Upper middle income,...,3-5 days,200.0,200.0,AUD,12.12,1.83024,yes,24/Jan/2011,AUSFJI,HSBC AUSFJI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205174,1120234566,2023_3Q,Malaysia,East Asia & Pacific,Upper middle income,IBRD,..,Thailand,East Asia & Pacific,Upper middle income,...,Less than one hour,610.0,200.0,MYR,9.93,7.59760,yes,12/Sep/2023,MYSTHA,MoneyGram MYSTHA
205175,1120237456,2023_3Q,Malaysia,East Asia & Pacific,Upper middle income,IBRD,..,Indonesia,East Asia & Pacific,Upper middle income,...,Less than one hour,610.0,200.0,MYR,2.85,3279.83320,yes,12/Sep/2023,MYSIDN,Western Union MYSIDN
205176,1120231791,2023_3Q,Australia,..,High income,..,G20,Vietnam,East Asia & Pacific,Lower middle income,...,2 days,200.0,200.0,AUD,8.99,15499.91630,yes,12/Sep/2023,AUSVNM,Commonwealth Bank AUSVNM
205177,1120231823,2023_3Q,Australia,..,High income,..,G20,Vanuatu,East Asia & Pacific,Lower middle income,...,2 days,200.0,200.0,AUD,4.16,77.82340,yes,12/Sep/2023,AUSVUT,Commonwealth Bank AUSVUT


In [165]:
corridors_present

['AREEGY',
 'AREIND',
 'ARELKA',
 'ARENPL',
 'AREPAK',
 'AREPHL',
 'AUSCHN',
 'AUSFJI',
 'AUSIND',
 'AUSPAK',
 'AUSPHL',
 'AUSTON',
 'AUSVNM',
 'AUSVUT',
 'AUSWSM',
 'AUTBIH',
 'AUTHRV',
 'AUTHUN',
 'AUTSRB',
 'AUTTUR',
 'BELCOD',
 'BELMAR',
 'BELTUR',
 'BRABOL',
 'BRAPER',
 'BRAPRY',
 'CANCHN',
 'CANGHA',
 'CANHTI',
 'CANIND',
 'CANJAM',
 'CANKEN',
 'CANPHL',
 'CANRWA',
 'CANVNM',
 'CANZWE',
 'CHELKA',
 'CHESRB',
 'CHLPER',
 'CRINIC',
 'CZEUKR',
 'CZEVNM',
 'DEUBIH',
 'DEUCHN',
 'DEUGHA',
 'DEUHRV',
 'DEUIND',
 'DEULBN',
 'DEUMAR',
 'DEUMDA',
 'DEUNGA',
 'DEUROU',
 'DEUSRB',
 'DEUTGO',
 'DEUTUR',
 'DOMHTI',
 'ESPBGR',
 'ESPBRA',
 'ESPCHN',
 'ESPCOL',
 'ESPDOM',
 'ESPECU',
 'ESPMAR',
 'ESPPER',
 'ESPPHL',
 'ESPROU',
 'FRACHN',
 'FRACIV',
 'FRADZA',
 'FRAHTI',
 'FRAIND',
 'FRAMAR',
 'FRAMLI',
 'FRASEN',
 'FRATGO',
 'FRATUN',
 'FRAVNM',
 'GBRALB',
 'GBRBGD',
 'GBRBGR',
 'GBRBRA',
 'GBRCHN',
 'GBRETH',
 'GBRGHA',
 'GBRGMB',
 'GBRIND',
 'GBRJAM',
 'GBRKEN',
 'GBRLKA',
 'GBRLTU',
 'GBRNGA',

In [166]:
unique_bank_df

,corridor,period,firm
62,AREEGY,2011_1Q,7
63,AREEGY,2011_3Q,7
64,AREEGY,2012_1Q,7
65,AREEGY,2012_3Q,7
66,AREEGY,2013_1Q,7
...,...,...,...
14758,ZAFZWE,2022_3Q,12
14759,ZAFZWE,2022_4Q,12
14760,ZAFZWE,2023_1Q,12
14761,ZAFZWE,2023_2Q,12


In [14]:
country_characters

,source_region,source_G8G20,destination_region,destination_G8G20,corridor
0,..,G20,East Asia & Pacific,..,AUSFJI
37,..,G20,South Asia,..,AUSPAK
82,..,G20,East Asia & Pacific,..,AUSWSM
120,..,G20,East Asia & Pacific,..,AUSTON
159,..,G20,East Asia & Pacific,..,AUSVUT
...,...,...,...,...,...
9192,..,..,East Asia & Pacific,..,NZLPHL
9286,..,..,East Asia & Pacific,..,NZLVNM
10646,..,G8/G20,Sub-Saharan Africa,..,USALBR
11382,..,G8/G20,Sub-Saharan Africa,..,FRATGO


In [168]:
hdi_gnipc

,iso3,hdi_2011,hdi_2012,hdi_2013,hdi_2014,hdi_2015,hdi_2016,hdi_2017,hdi_2018,hdi_2019,...,gnipc_2013,gnipc_2014,gnipc_2015,gnipc_2016,gnipc_2017,gnipc_2018,gnipc_2019,gnipc_2020,gnipc_2021,gnipc_2022
0,AFG,0.457,0.467,0.475,0.480,0.479,0.483,0.485,0.486,0.492,...,2244.209474,2222.094339,2142.689163,2122.363226,2123.676477,2082.113799,2112.986381,1986.803772,1534.135377,1335.205733
1,ALB,0.779,0.789,0.793,0.797,0.797,0.795,0.796,0.797,0.800,...,11552.966170,11691.631800,12016.280650,12484.606590,12802.175940,13302.733030,13481.971800,13069.127700,14399.780470,15293.326510
2,DZA,0.725,0.726,0.729,0.732,0.736,0.738,0.738,0.740,0.742,...,11118.248780,11289.719290,11475.065750,11768.622670,11633.272120,11438.083580,11353.521810,10634.883980,10823.118130,10978.405710
3,AND,0.867,0.867,0.851,0.854,0.856,0.859,0.860,0.863,0.865,...,52499.189810,53623.112470,54297.459370,55695.331800,54906.348230,54904.290060,55035.017860,48026.390490,51128.347500,54233.449480
4,AGO,0.533,0.545,0.555,0.565,0.591,0.595,0.597,0.598,0.597,...,7437.535052,7651.196992,7585.829435,7111.865908,6772.546424,6310.063799,5918.032996,5487.669515,5332.948130,5327.788251
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,VEN,0.766,0.769,0.771,0.768,0.766,0.759,0.750,0.738,0.720,...,19973.370000,18928.600000,17511.156000,14482.920000,12322.101000,10427.235000,7836.525000,5459.720000,5587.197000,6184.136000
191,VNM,0.681,0.684,0.689,0.693,0.697,0.702,0.706,0.711,0.717,...,7020.053368,7361.555222,7697.652174,8084.754808,8503.905970,9141.841879,9733.786723,10005.174610,10084.943720,10813.982730
192,YEM,0.495,0.497,0.496,0.485,0.455,0.443,0.432,0.427,0.430,...,2994.007003,2582.869899,1659.403107,1386.769295,1207.062488,1170.253731,1165.090740,1152.015672,1112.162989,1105.763435
193,ZMB,0.534,0.548,0.554,0.557,0.563,0.564,0.568,0.571,0.574,...,3320.860732,3192.959658,3326.211349,3159.451506,3244.979459,3325.308024,3270.200936,3046.055722,3098.623658,3157.358620


So far